##### https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

In [2]:
import pandas as pd
import numpy as np


from sklearn.svm import SVC
import matplotlib.pyplot as plt


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import cross_val_score


In [3]:
df = pd.read_excel('Part 3 -  Absenteeism_at_work.xls')
print(df.shape)


(741, 21)


In [ ]:
print(df.isna().sum())
print(df.nunique())
print(df.head(),df.tail())

In [4]:
#drop last row = all null
df.drop(740,inplace=True)
df.describe().T
df.shape

(740, 21)

In [ ]:
X = df.drop('Absenteeism time in hours',axis=1)
y = df['Absenteeism time in hours']

X_train,X_test,y_train ,y_test = train_test_split(X,y,random_state = 42,train_size = 0.8,test_size=0.2)
#X_train,X_test ,y_train ,y_test

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train,y_train)


# Make predictions using the testing set
y_pred = lin_model.predict(X_test)
y_pred_train = lin_model.predict(X_train)

print('train root Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_train, y_pred_train)))

print('test Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_test, y_pred)))
lin_model.score(X_train,y_train)


In [6]:
df_duplicated = df.copy()
display(df)
for idx, row in df.iterrows():
    hours = row.loc['Absenteeism time in hours']
    if hours > 8:
        days = int(hours/8)
        row_tmp = row
        row_tmp.loc['Absenteeism time in hours'] = 8
        day_of_week = row.loc['Day of the week'] 
        for i in range(days):
            row_tmp.loc['Day of the week']  = day_of_week 
            df_duplicated = df_duplicated.append([row_tmp],ignore_index=True)
            if (day_of_week == 6):
                day_of_week = 2
            else:
                day_of_week += 1
        df_duplicated.drop(idx,inplace=True)

#print(df_duplicated.shape)
display(df_duplicated)

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11.0,26.0,7.0,3.0,1.0,289.0,36.0,13.0,33.0,239554.0,...,0.0,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,4
1,36.0,0.0,7.0,3.0,1.0,118.0,13.0,18.0,50.0,239554.0,...,1.0,1.0,1.0,1.0,0.0,0.0,98.0,178.0,31.0,0
2,3.0,23.0,7.0,4.0,1.0,179.0,51.0,18.0,38.0,239554.0,...,0.0,1.0,0.0,1.0,0.0,0.0,89.0,170.0,31.0,2
3,7.0,7.0,7.0,5.0,1.0,279.0,5.0,14.0,39.0,239554.0,...,0.0,1.0,2.0,1.0,1.0,0.0,68.0,168.0,24.0,4
4,11.0,23.0,7.0,5.0,1.0,289.0,36.0,13.0,33.0,239554.0,...,0.0,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,11.0,14.0,7.0,3.0,1.0,289.0,36.0,13.0,33.0,264604.0,...,0.0,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,8
736,1.0,11.0,7.0,3.0,1.0,235.0,11.0,14.0,37.0,264604.0,...,0.0,3.0,1.0,0.0,0.0,1.0,88.0,172.0,29.0,4
737,4.0,0.0,0.0,3.0,1.0,118.0,14.0,13.0,40.0,271219.0,...,0.0,1.0,1.0,1.0,0.0,8.0,98.0,170.0,34.0,0
738,8.0,0.0,0.0,4.0,2.0,231.0,35.0,14.0,39.0,271219.0,...,0.0,1.0,2.0,1.0,0.0,2.0,100.0,170.0,35.0,0


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11.0,26.0,7.0,3.0,1.0,289.0,36.0,13.0,33.0,239554.0,...,0.0,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,4.0
1,36.0,0.0,7.0,3.0,1.0,118.0,13.0,18.0,50.0,239554.0,...,1.0,1.0,1.0,1.0,0.0,0.0,98.0,178.0,31.0,0.0
2,3.0,23.0,7.0,4.0,1.0,179.0,51.0,18.0,38.0,239554.0,...,0.0,1.0,0.0,1.0,0.0,0.0,89.0,170.0,31.0,2.0
3,7.0,7.0,7.0,5.0,1.0,279.0,5.0,14.0,39.0,239554.0,...,0.0,1.0,2.0,1.0,1.0,0.0,68.0,168.0,24.0,4.0
4,11.0,23.0,7.0,5.0,1.0,289.0,36.0,13.0,33.0,239554.0,...,0.0,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,13.0,13.0,7.0,2.0,1.0,369.0,17.0,12.0,31.0,264604.0,...,0.0,1.0,3.0,1.0,0.0,0.0,70.0,169.0,25.0,8.0
979,13.0,13.0,7.0,3.0,1.0,369.0,17.0,12.0,31.0,264604.0,...,0.0,1.0,3.0,1.0,0.0,0.0,70.0,169.0,25.0,8.0
980,13.0,13.0,7.0,4.0,1.0,369.0,17.0,12.0,31.0,264604.0,...,0.0,1.0,3.0,1.0,0.0,0.0,70.0,169.0,25.0,8.0
981,13.0,13.0,7.0,5.0,1.0,369.0,17.0,12.0,31.0,264604.0,...,0.0,1.0,3.0,1.0,0.0,0.0,70.0,169.0,25.0,8.0


In [7]:
X_duplicated = df_duplicated.drop(columns=['ID','Absenteeism time in hours'],axis=1)
y_duplicated = df_duplicated['Absenteeism time in hours']

display(X_duplicated.shape)

X_train,X_test,y_train ,y_test = train_test_split(X_duplicated,y_duplicated,random_state = 42,train_size = 0.8,test_size=0.2)
#X_train,X_test ,y_train ,y_test
lin_model = LinearRegression()
lin_model.fit(X_train,y_train)


# Make predictions using the testing set
y_pred = lin_model.predict(X_test)
y_pred_train = lin_model.predict(X_train)

print('train root Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_train, y_pred_train)))

print('test Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_test, y_pred)))

lin_model.score(X_test,y_test)


(982, 19)

train root Mean squared error: 10.65
test Mean squared error: 11.61


0.057927498800881105